In [1]:
import pandas as pd

## Задание 1

In [12]:
table_rating = pd.read_csv('files/ratings1.csv')
table_name = pd.read_csv('files/movies1.csv')

In [13]:
table_rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
table_name.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
def rating_recognition(row):
    if row <= 2:
        return "Низкий рейтинг"
    elif row <= 4:
        return "Средний рейтинг"
    elif row == 4.5 or row == 5:
        return "Высокий рейтинг"
    else:
        return "Недопустимое значение рейтинга"

In [17]:
print(rating_recognition(2))

Низкий рейтинг


In [20]:
table_name['rating_string'] = table_rating['rating'].apply(rating_recognition)
table_name.head()

,movieId,title,genres,rating_string
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Низкий рейтинг
4,5,Father of the Bride Part II (1995),Comedy,Средний рейтинг


## Задание 2

In [2]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],

            'Северо-Запад': ['петербург', 'псков', 'мурманск'],

            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
            }

In [3]:
table_search = pd.read_csv('files/keywords.csv')

#### Вариант менее правильный(просто первый который заработал) 
не мог понять как работает loc оч долго, хотел было в вопрос выносить, но пока писал озарило))

In [4]:
def return_region(row, geo_data=geo_data):
    for region, cities in geo_data.items():
        if bool(set(row.split(' ')) & set(cities)):
            return region
    return 'undefined'
            

In [5]:
%%time
table_search['region'] = table_search['keyword'].apply(return_region)

Wall time: 246 ms


In [6]:
table_search['region'].value_counts()

undefined         99406
Центр               384
Северо-Запад        146
Дальний Восток       64
Name: region, dtype: int64

#### Вариант с большим нахождением - конечный (учтены склонения и регистр)

In [8]:
%%time
table_search['region'] = 'undefined'
for region, cities in geo_data.items():
    pattern = '|'.join(cities)
    table_search.loc[table_search.keyword.str.contains(pattern, case=False), 'region'] = region

Wall time: 307 ms


In [9]:
table_search.region.value_counts()

undefined         99253
Центр               385
Северо-Запад        268
Дальний Восток       94
Name: region, dtype: int64

## Вопрос
Пытался через lambda сделать но ругается на синтаксис, парность скобок проверил, и все равно...

In [10]:
towns = ['москва', 'тула', 'ярославль']
table_search['region'] = table_search['keyword'].apply(lambda row: 'центр' if bool(set(row.split(' ')) & set(towns)))

SyntaxError: invalid syntax (<ipython-input-10-a24453314e5b>, line 2)

## Задание 3

In [27]:
table_rating = pd.read_csv('files/ratings1.csv')
table_name = pd.read_csv('files/movies1.csv')

In [28]:
years = []
for year in range(1950,2011,1):
    years.append(str(year))

In [29]:
def production_year(row):
    for year in years:
        if year in row.title:
            return year
    return '1900'
        

In [30]:
%%time
table_name['year'] = table_name.apply(production_year, axis=1)
table_name

Wall time: 3.4 s


,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,1900
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,1900
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,1900
9123,164977,The Gay Desperado (1936),Comedy,1900


In [45]:
table_id_year = table_name[['movieId', 'year']]

In [46]:
table_id_year.groupby('year').apply(lambda row: row.movieId.mean()).reset_index().sort_values(0, ascending=False).rename(columns={0:'mean_rating'})

,year,mean_rating
0,1900,84100.316770
61,2010,82593.271930
60,2009,74637.220472
59,2008,66353.473684
58,2007,59276.257812
...,...,...
48,1997,4617.168539
45,1994,4446.150000
32,1981,4232.535354
47,1996,2713.007273
